# CentraleSupelec - Natural language processing
# Practical session n°7

## Natural Language Inferencing (NLI): 

(NLI) is a classical NLP (Natural Language Processing) problem that involves taking two sentences (the premise and the hypothesis ), and deciding how they are related- if the premise *entails* the hypothesis, *contradicts* it, or *neither*.

Ex: 


| Premise | Label | Hypothesis |
| --- | --- | --- |
| A man inspects the uniform of a figure in some East Asian country. | contradiction | The man is sleeping. |
| An older and younger man smiling. | neutral | Two men are smiling and laughing at the cats playing on the floor. |
| A soccer game with multiple males playing. | entailment | Some men are playing a sport. |

### Stanford NLI (SNLI) corpus

In this labwork, I propose to use the Stanford NLI (SNLI) corpus ( https://nlp.stanford.edu/projects/snli/ ), available in the *Datasets* library by Huggingface.

    from datasets import load_dataset
    snli = load_dataset("snli")
    #Removing sentence pairs with no label (-1)
    snli = snli.filter(lambda example: example['label'] != -1) 

## Subject

You are asked to provide an operational Jupyter notebook that performs the task of NLI. For that, you need to tackle the following aspects of the problem:

1. Loading and preprocessing the data
2. Designing a PyTorch model that, given two sentences, decides how they are related (*entails*, *contradicts* or *neither*.)
3. Training and evaluating the model using appropriate metrics
4. (Optional) Allowing to play with the model (forward user sentences and visualize the prediction easily)
5. (Optional) Providing visual insight about the model (i.e. visualizing the attention if your model is using attention)

Although it is not mandatory, I suggest that you use a transformer model to perform the task. For that, you can use the *Transformer* library by Huggingface.

## Evaluation

The evaluation will be based on several criteria:

- Clarity and readability of the notebook. The notebook is the report of you project. Make it easy and pleasant to read.
- Justification of implementation choices (i.e. the network, the cost funtion, the optimizer, ...)
- Quality of the code. The various deeplearning and NLP labworks provide many example of good practices for designing experiments with neural networks. Use them as inspirational examples!

## Additional recommendations

- You are not seeking to publish a research paper! I'm not expecting state-of-the-art results! The idea of this labwork is to assess that you have integrated the skills necessary to handle textual data using deep neural network techniques.

- This labwork will be evaluated but we are still here to help you! Don't hesitate to request our help if you are stuck.

- If you intend to use BERT based models, let me give you an advice. The bert-base-* models available in *Transformers* need more than 12Go to be fine-tuned on GPU. To avoid memory issues, you can use several solutions: 

    - Use a lighter BERT based model such as DistilBERT, ALBERT, ...
    - Train a classification model on top of BERT, whithout fine-tuning it (i.e. freezing BERT weights)

## Huggingface documentations

In case you want to use the huggingface *Datasets* and *Transformer* libraries (which I advice), here are some useful documentation pages:

- Dataset quick tour

    https://huggingface.co/docs/datasets/quicktour.html
    
- Documentation on data preprocessing for transformers

    https://huggingface.co/transformers/preprocessing.html
    
- Transformer Quick tour (with distilbert example for classification).

    https://huggingface.co/transformers/quicktour.html
    


## Part 0 : Imports

In [ ]:
from nltk.tokenize import word_tokenize 
import pandas as pd
import os
from datasets import load_dataset
from transformers import AutoTokenizer, BertForSequenceClassification,AutoModelForSequenceClassification, DistilBertConfig, DistilBertTokenizer, DistilBertForSequenceClassification, BertTokenizer, BertModel
import time
import multiprocessing
from tqdm import tqdm
import torch
from torch import nn

## Part 0 bis : Variables

In [ ]:
BATCH_SIZE = 64
device = 'cuda' if torch.cuda.is_available() else 'cpu'
learning_rate = 1e-5
epocs = 10

## Part 1 : Load data / tokenizer and preprocess


Dans cette première partie, on va télécharger le dataset et le stocker dans un dataframe pandas. Ensuite, on effectura un preprocessing des données en effectuant une tokenization des phrases des corpus.

In [ ]:
snli = load_dataset("snli")
#Removing sentence pairs with no label (-1)
snli = snli.filter(lambda example: example['label'] != -1)

In [ ]:
snli

In [ ]:
snli['train']['premise'][1]

In [ ]:
snli['train']['hypothesis'][1]

In [ ]:
snli['train']['label'][1]

In [ ]:
train = snli['train']
validation = snli['validation']
test = snli['test']

In [ ]:
print(train.shape)
print(validation.shape)
print(test.shape)

In [ ]:
from transformers import DistilBertConfig, DistilBertTokenizer, DistilBertForSequenceClassification

bertmodel = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",output_attentions=True,num_labels=3)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
tokenizer

In [ ]:
def bert_encode(dataset):
    return(tokenizer(dataset['premise'], dataset['hypothesis'], truncation=True, padding='max_length',max_length = 110))

In [ ]:
train = train.map(bert_encode, batched=True,batch_size = BATCH_SIZE, num_proc=os.cpu_count())
test = test.map(bert_encode, batched=True,batch_size = BATCH_SIZE,num_proc=os.cpu_count())
validation = validation.map(bert_encode, batch_size = BATCH_SIZE,batched=True,num_proc=os.cpu_count())

In [ ]:
print(train[0])

In [ ]:
train = train.map(lambda examples: {'labels': examples['label']}, batched=True, num_proc = os.cpu_count())
validation = validation.map(lambda examples: {'labels': examples['label']}, batched=True, num_proc = os.cpu_count())
test = test.map(lambda examples: {'labels': examples['label']}, batched=True, num_proc = os.cpu_count())

In [ ]:
train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
validation.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

train_dataloader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, num_workers=os.cpu_count())
val_dataloader = torch.utils.data.DataLoader(validation, batch_size=BATCH_SIZE,num_workers=os.cpu_count())
test_dataloader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE,num_workers=os.cpu_count())

In [ ]:
next(iter(train_dataloader))

In [ ]:
print(train['premise'][1])
print(tokenizer.decode(train['input_ids'][1]))

Entrainons maintenant le modèle Bert large cased

## Part 2 : Train / Evaluation

In [ ]:
def train_bert(model,clip):
    totloss=0
    model.train().to(device)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    for i, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs[0]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        totloss+= loss.item()
        optimizer.step()
        optimizer.zero_grad()
        if i % 100 == 0:
            print(f"train_loss: {loss}")
    lossavg = totloss/len(train_dataloader)
    print(f"Loss: {lossavg}")
    return lossavg

def val(model):
    model.train().to(device)    
    model.eval()
    totloss = 0
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    for i, batch in enumerate(tqdm(val_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs[0]
        totloss+=loss.item()
        if i % 100 == 0:
            print(f"Val loss: {loss:.3f}")
    lossavg = totloss/len(train_dataloader)
    print(f"Loss: {lossavg}")
    return lossavg


In [ ]:
#https://github.com/12860/NLP/blob/master/Seq2seq.py

best_validation_loss = float('inf')
clip=1
for epoch in range(epocs):
    print('Epoch : ',epoch)    
    train_loss = train_bert(bertmodel,clip)
    valid_loss = val(bertmodel)
    if valid_loss < best_validation_loss:
        print('Saving a new model')
        best_valid_loss = valid_loss
        torch.save(bertmodel.state_dict(), '/usr/users/gpusdi1/gpusdi1_26/Documents/NLP/NLP_Natural-Language-Inferencing/dist_model.pt')
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f}')

## Part 3 : Testing the model ##

In [ ]:
torch.load('/usr/users/gpusdi1/gpusdi1_26/Documents/NLP/NLP_Natural-Language-Inferencing/dist_model.pt')

In [ ]:
def test_model(model,dataloader):
    model.train().to(device)    
    model.eval()
    for i, batch in enumerate(tqdm(dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)

In [ ]:
test_model(bertmodel,test_dataloader)

## Part 4 : Using our sentences ##

## Part 3 : Val

In [ ]:
def val():
    model.eval()
    totloss = 0
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    for i, batch in enumerate(tqdm(val_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs[0]
        total_loss+=loss.item()
        if i % 100 == 0:
            print(f"Val loss: {loss:.3f}")
    lossavg = totloss/len(train_dataloader)
    print(f"Loss: {lossavg}")
    return lossavg

In [ ]:
val()